In [65]:
import pandas as pd
import numpy as np

In [66]:
df = pd.read_csv('/Users/nehanegi/Desktop/Fall 2022/Distributed Information Systems/expense_tracker/data/expenses.csv',index_col="id")

In [67]:
df.head()

,expense_date,category,amount
id,,,
1,1/1/2022,Shopping,11.11
2,1/2/2022,Mortgage & Rent,1247.44
3,1/2/2022,Food & Dining,24.22
4,1/4/2022,Entertainment,11.76
5,1/5/2022,Food & Dining,25.85


In [68]:
df.dtypes

expense_date     object
category         object
amount          float64
dtype: object

In [69]:
df['expense_date']=pd.to_datetime(df['expense_date'],infer_datetime_format=True)

In [70]:
df.dtypes

expense_date    datetime64[ns]
category                object
amount                 float64
dtype: object

In [71]:
df.head()

,expense_date,category,amount
id,,,
1,2022-01-01,Shopping,11.11
2,2022-01-02,Mortgage & Rent,1247.44
3,2022-01-02,Food & Dining,24.22
4,2022-01-04,Entertainment,11.76
5,2022-01-05,Food & Dining,25.85


In [72]:
df['Month'] = df['expense_date'].dt.strftime('%b')

In [73]:
df.head()

,expense_date,category,amount,Month
id,,,,
1,2022-01-01,Shopping,11.11,Jan
2,2022-01-02,Mortgage & Rent,1247.44,Jan
3,2022-01-02,Food & Dining,24.22,Jan
4,2022-01-04,Entertainment,11.76,Jan
5,2022-01-05,Food & Dining,25.85,Jan


In [74]:
data = df[['category','amount','Month']]

In [75]:
data.head()

,category,amount,Month
id,,,
1,Shopping,11.11,Jan
2,Mortgage & Rent,1247.44,Jan
3,Food & Dining,24.22,Jan
4,Entertainment,11.76,Jan
5,Food & Dining,25.85,Jan


## Split the data into train and test

In [76]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.3)

## Data Prep

In [77]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [78]:
train_targets = train[['amount']]
test_targets = test[['amount']]

train_inputs = train.drop(['amount'], axis=1)
test_inputs = test.drop(['amount'], axis=1)

In [79]:
train_inputs.dtypes

category    object
Month       object
dtype: object

In [80]:
categorical_columns = ['category','Month']

In [81]:
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [82]:
preprocessor = ColumnTransformer([('cat', categorical_transformer, categorical_columns)],
        remainder='passthrough')

In [83]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

<288x22 sparse matrix of type '<class 'numpy.float64'>'
	with 576 stored elements in Compressed Sparse Row format>

In [84]:
train_x.shape

(288, 22)

In [85]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

<124x22 sparse matrix of type '<class 'numpy.float64'>'
	with 248 stored elements in Compressed Sparse Row format>

In [86]:
test_x.shape

(124, 22)

## Linear Regression Model

In [87]:

from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

lin_reg.fit(train_x, train_targets)

LinearRegression()

In [88]:
from sklearn.metrics import mean_squared_error

In [89]:
#Train RMSE
reg_train_pred = lin_reg.predict(train_x)

train_mse = mean_squared_error(train_targets, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 74.82139819291213


In [90]:
#Test RMSE
reg_test_pred = lin_reg.predict(test_x)

test_mse = mean_squared_error (test_targets, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 720.8679545629872
